<div style="display: flex; background-color: skyblue;" >
<h1 style="margin: auto; padding: 30px; "> P12 -NB2-APPLI-  
    - Détection de faux billets – Projet de modélisation</h1></div>

**Vérification de l'environnement :**

In [42]:
!where python

D:\Programmes2\Anaconda3\python.exe
C:\Users\matth\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\matth\AppData\Local\Microsoft\WindowsApps\python.exe


<u>**Remarque :**</u>  
Pour des raisons de lisibilité des parties ont été masquées.
Elles peuvent être démasquées directement sur le Notebook .ipynb

**CE NOTEBOOK EST LA PARTIE APPLICATION DU PROJET**

##### <font color='blue'><u>**SUJET :**</u></font>  
- **Projet 12, dans le cadre de la formation Data Analyst de OpenClassRooms**
- **Enoncé du projet :** https://openclassrooms.com/fr/paths/804/projects/1584
- **Objectifs pédagogiques :**  
      . Utiliser un modèle d'apprentissage supervisé pour réaliser une prédiction d'un phénomène  
      . Entraîner un modèle d'apprentissage non supervisé pour exploiter le modèle

##### <font color='blue'><u>**PRESENTATION RESUMEE :**</u></font>   

<u>**ENTREPRISE :**</u>   
**Nom :** l’Organisation nationale de lutte contre le faux-monnayage (ONCFM)  
**Secteur :** Organisation publique  
**Activité :** Identification des contrefaçons de billets en euros    

<u>**BESOIN :**</u>  
mettre à disposition des équipes un algorithme capable de faire une prédiction sur la nature des billets (vrai billet ou faux billet).  

<u>**Objectif :**</u>  
-	Mettre en place application de machine learning  
-	respectant le cahier des charges fourni, notamment  
-	Entrée : caractéristiques géométriques d’un billet  
-	Sortie : authenticité d’un billet  


##### <font color='blue'><u>**PRESENTATION DETAILLEE :**</u></font>   

<u>**Contexte :**</u>  

Dans le cadre de ses actions de lutte contre la contrefaçon, l’**Organisation nationale de lutte contre le faux-monnayage (ONCFM)** souhaite automatiser la détection des faux billets à partir de caractéristiques géométriques mesurées lors du passage en machine.

L’ONCFM a confié à notre entreprise le développement d’un **prototype de modèle prédictif** permettant de distinguer automatiquement les vrais billets des faux.  
Cette détection repose exclusivement sur des données quantitatives relatives à la forme du billet (longueur, hauteur, marges, diagonale...).

<u>**Objectif de la mission :**</u>  

En tant que **Data Analyst en charge du projet**, ma mission consiste à :

- Réaliser une **analyse exploratoire** des données fournies (1500 billets : 1000 vrais et 500 faux) ;
- Appliquer différents **modèles de machine learning supervisés et non supervisés** (logistic regression, K-means, KNN, random forest) pour évaluer leur capacité de détection ;
- Mettre en place un **prototype d'application fonctionnelle** capable de prédire la nature d’un billet à partir de ses caractéristiques ;
- Documenter les résultats et **justifier le choix du modèle final** retenu, selon sa robustesse, sa performance et sa facilité d'interprétation ;
- Proposer des recommandations métier à l’ONCFM pour l’usage opérationnel de l’outil.

<u>**Données :**</u>  

Le fichier fourni contient les caractéristiques suivantes pour chaque billet :
- `length` : longueur (mm)
- `height_left` : hauteur côté gauche (mm)
- `height_right` : hauteur côté droit (mm)
- `margin_up` : marge supérieure (mm)
- `margin_low` : marge inférieure (mm)
- `diagonal` : diagonale (mm)
- `is_genuine` : étiquette indiquant si le billet est un vrai (1) ou un faux (0)

---

L’approche adoptée sera documentée pas à pas dans ce Notebook, depuis l’analyse initiale jusqu’au choix du modèle final.


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 1 - Import des librairies et chargement des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.1 - Import des librairies</h3>
</div>

In [43]:
#Import de la librairie Numpy
import numpy as np

In [44]:
#Import de la librairie Pandas
import pandas as pd

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Import du modèle entrainé (+choix)et du scaler entrainé </h3>
</div>

In [45]:
import joblib

#IMPORT DU MODEL SELECTIONNE
random_forest_model = joblib.load(r"D:\BDD\OCR\P12\P12_Sorties\random_forest_model.pkl")

In [46]:
#OPTION 2 
#IMPORT DU MODEL SELECTIONNE
regression_Logistique_model = joblib.load(r"D:\BDD\OCR\P12\P12_Sorties\regression_Logistique_model.pkl")

**CHOIX DU MODELE :**

In [47]:
#CHOIX DU MODELE :

model_final = random_forest_model

#model_final = regression_Logistique_model

**IMPORT du scaler préfitté avec X_train :**

In [48]:
# Chargement du scaler
scaler_prefit = joblib.load(r"D:\BDD\OCR\P12\P12_Sorties\standard_scaler_prefit_X_train.pkl")

« Le même StandardScaler, ajusté sur les données d'entraînement initiales, a été enregistré et rechargé dans l'application afin de garantir une standardisation cohérente des nouvelles données. Cela permet d’éviter tout biais introduit par un réajustement sur des données différentes. »

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.3 - Import du Fichier de Billets à Analyser</h3>
</div>

##### <font color='blue'>**Import du fichier billets_production Test**</font>

In [49]:
#Import du fichier billets Test
df_billets_production_Test = pd.read_csv(r"D:\BDD\OCR\P12\P12_original\billets_production.csv", sep = ',')
df_billets_production_Test.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


##### <font color='blue'>**Import du fichier billets_production Soutenance**</font>  
<font color ='orange'>**CHEMIN A COMPLETER LORS DE LA SOUTENANCE**</font>

In [50]:
#Import du fichier billets Soutenance

#EN ATTENTE DU FICHIER/CHEMIN A REMPLIR : fichier à déposer dans D:\BDD\OCR\P12\P12_Soutenance\

#df_billets_production_Soutenance = pd.read_csv(r"D:\BDD\OCR\P12\P12_Soutenance\_NOM_FICHIER_.csv", sep = ',')

#df_billets_production_Soutenance = pd.read_csv(r"D:\BDD\OCR\P12\P12_original\billets_test_blanc.csv", sep = ',')

##### **CHOIX DU FICHIER A TESTER**  
<font color ='orange'>**CHOIX A MODIFIER LORS DE LA SOUTENANCE**</font>

In [51]:
#CHOIX DU FICHIER A TESTER

df_a_tester = df_billets_production_Test.copy(deep=True)

#df_a_tester = df_billets_production_Soutenance.copy(deep=True)

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 2 - Elaboration des Résultats :</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 -  Préparation des données TEST : </h3>
</div>

In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#listes des variables prédictives pour le modèle RL_final : toutes les var quantitatives
var_pred = ['diagonal',
            'height_left', 'height_right', 
            'margin_low','margin_up', 
            'length']
#la variable cible :
var_cible = 'is_genuine'

#le jeu de données prédictif complet:
X_test = (df_a_tester[var_pred]).copy(deep=True)

In [53]:
print(X_test)

   diagonal  height_left  height_right  margin_low  margin_up  length
0    171.76       104.01        103.54        5.21       3.30  111.42
1    171.87       104.17        104.13        6.00       3.31  112.09
2    172.00       104.58        104.29        4.99       3.39  111.57
3    172.49       104.55        104.34        4.44       3.03  113.20
4    171.65       103.63        103.56        3.77       3.16  113.33


**standardisation des données tests avec le scaler prefitté et entrainé avec X_train utilisé pour le modèle dans le NB1**

In [54]:
#standardisation des données avec le scaler prefitté
X_test_scaled = scaler_prefit.transform(X_test)

In [55]:
print(X_test_scaled)

[[-0.64127206 -0.06756023 -1.164697    1.1207046   0.64330831 -1.44521223]
 [-0.2833225   0.46384348  0.64384665  2.31990014  0.68646406 -0.68281745]
 [ 0.1397088   1.82556548  1.13429916  0.78675141  1.03171002 -1.27452683]
 [ 1.73421139  1.72592728  1.28756558 -0.04813156 -0.52189681  0.58025449]
 [-0.99922162 -1.32964404 -1.10339044 -1.06517081  0.03912788  0.72818184]]


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.2 -  Prédictions & Probabilités : </h3>
</div>

In [56]:
y_test_pred = model_final.predict(X_test_scaled)

In [57]:
print(y_test_pred)

[0 0 0 1 1]


In [58]:
# Probabilités (classe 1)
y_test_proba = model_final.predict_proba(X_test_scaled)[:, 1]

In [59]:
print(y_test_proba)

[0.012 0.    0.    0.982 1.   ]


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.3 -  ELABORATION DU RESULTAT : </h3>
</div>

In [60]:
df_resultat = df_a_tester.copy(deep=True)

In [61]:
pd.DataFrame(y_test_pred).astype(bool)

,0
0,False
1,False
2,False
3,True
4,True


In [62]:
df_resultat['Authenticité'] = pd.DataFrame(y_test_pred).astype(bool)
df_resultat['Auth_Proba_%'] = (pd.DataFrame(y_test_proba))*100

In [63]:
df_resultat.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id,Authenticité,Auth_Proba_%
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,1.2
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.0
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.0
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,98.2
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,100.0


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 3 - AFFICHAGE DU RESULTAT </h2>
</div>

In [64]:
print("RESULTAT SIMPLE :")
display(df_resultat[['id','Authenticité', 'Auth_Proba_%']])

RESULTAT SIMPLE :


,id,Authenticité,Auth_Proba_%
0,A_1,False,1.2
1,A_2,False,0.0
2,A_3,False,0.0
3,A_4,True,98.2
4,A_5,True,100.0


In [65]:
print("RESULTAT COMPLETS :")
display(df_resultat)

RESULTAT COMPLETS :


,diagonal,height_left,height_right,margin_low,margin_up,length,id,Authenticité,Auth_Proba_%
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,False,1.2
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,False,0.0
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,False,0.0
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,True,98.2
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,True,100.0


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 4 - EXPORT DES RESULTATS  </h2>
</div>

In [66]:
#EXPORT DES RESULTATS :

df_resultat.to_csv(r"D:\BDD\OCR\P12\P12_Sorties\Resultats_Authenticite_AAMMDDHHMM.csv",index=False)